# 🍠 [Day 18](https://adventofcode.com/2019/day/18)

In [1]:
import heapq
import numpy as np
from collections import defaultdict

def parse_maze(inputs):
    maze = [list(line) for line in inputs.splitlines()]
    return np.array(maze)
    
        
def bfs_with_constraints(maze):
    start = np.where(maze == '@')
    num_keys = np.amax(list(map(ord, maze.flatten()))) - ord('a') + 1
    # Save minimum distances from starting point 
    # for every target point x keys combination
    dists = defaultdict(lambda: defaultdict(lambda: float('inf')))
    # Queue
    # cost, num keys left, keys found, current point
    # The heap will prioritize point with low cost 
    # and, at equal cost, highest number of keys
    queue = [(0, num_keys, (), start[0][0], start[1][0])]
    heapq.heapify(queue)
    
    # Output
    keys_order = None
    min_dist = float('inf')
    
    while len(queue):
        cost, k, keys, x, y = heapq.heappop(queue)
        if k != num_keys - len(keys):
            break
        for (i, j) in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
            # Wall or locked door
            if (maze[i, j] == '#' or 
                (ord('A') <= ord(maze[i, j]) < ord('a') 
                 and maze[i, j].lower() not in keys)):
                continue
            # Pick up a new key
            elif ord(maze[i, j]) >= ord('a') and maze[i, j] not in keys:
                # Update cost for new keys + next state
                new_keys = keys + (maze[i, j],)
                hashkey = tuple(sorted(new_keys))
                if dists[hashkey][(i, j)] > cost + 1:
                    dists[hashkey][(i, j)] = cost + 1
                    # Add to queue if still to be found
                    if k > 1:
                        heapq.heappush(queue, (cost + 1, k - 1, new_keys, i, j))
                    # Otherwise, save key order
                    elif cost + 1 < min_dist:
                        min_dist = cost + 1
                        keys_order = new_keys
            # Advance / normal path
            else:
                hashkey = tuple(sorted(keys))
                if dists[hashkey][(i, j)] > cost + 1:
                    dists[hashkey][(i, j)] = cost + 1
                    heapq.heappush(queue, (cost + 1, k, keys, i, j))
    return min_dist, keys_order


In [2]:
%%time
with open('inputs/day18.txt', 'r') as f:
    maze = parse_maze(f.read())
    
print("The shortest path takes a total of {0} steps and is "
      "achieved by collecting the keys in order {1}".format(
      *bfs_with_constraints(maze)))

The shortest path takes a total of 6162 steps and is achieved by collecting the keys in order ('e', 'f', 'z', 'b', 'i', 'l', 'a', 'u', 'p', 'd', 'c', 'k', 'o', 'r', 's', 'j', 'q', 'm', 't', 'v', 'x', 'h', 'g', 'n', 'y', 'w')
CPU times: user 16.3 s, sys: 151 ms, total: 16.4 s
Wall time: 16.4 s
